<a href="https://colab.research.google.com/github/kyangwi/CV/blob/main/Face_Recognition_using_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install dlib-bin
!pip install numpy scikit-learn opencv-python
!pip install insightface       # core library (ArcFace, RetinaFace models) :contentReference[oaicite:0]{index=0}
!pip install onnxruntime-gpu   # GPU inference backend :contentReference[oaicite:1]{index=1}


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 124.3 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1064781 sha256=a8719cf0ad4340433c69de8495584769709726aded6789ec0e5737b3a7eaf00e
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.8 MB/s eta 0:00:00


In [4]:
import os
import cv2
import dlib
import numpy as np
from sklearn.svm import SVC
import pickle

In [5]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

--2025-04-28 18:15:28--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-04-28 18:15:28--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  31.8MB/s    in 1.9s    

2025-04-28 18:15:30 (31.8 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [6]:
!wget https://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2

--2025-04-28 18:15:36--  https://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21428389 (20M)
Saving to: ‘dlib_face_recognition_resnet_model_v1.dat.bz2’

dlib_face_recogniti 100%[===================>]  20.44M  16.8MB/s    in 1.2s    

2025-04-28 18:15:37 (16.8 MB/s) - ‘dlib_face_recognition_resnet_model_v1.dat.bz2’ saved [21428389/21428389]



# **1. USING dLib embeddings 128 vector**

In [7]:
path = "/content/drive/MyDrive/facedata"

In [8]:
import bz2
comp = open("/content/shape_predictor_68_face_landmarks.dat.bz2", "rb").read()
with open("/content/drive/MyDrive/FaceRecognitionModels/shape_predictor_68_face_landmarks.dat", "wb") as f:
    f.write(bz2.decompress(comp))

In [9]:
import bz2
comp = open("/content/dlib_face_recognition_resnet_model_v1.dat.bz2", "rb").read()
with open("/content/drive/MyDrive/FaceRecognitionModels/dlib_face_recognition_resnet_model_v1.dat", "wb") as f:
    f.write(bz2.decompress(comp))

In [10]:
image_path = "/content/drive/MyDrive/facedata"

In [11]:
# 1. Load models
hog_detector = dlib.get_frontal_face_detector()
shape_pred = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")
face_rec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")


In [12]:
# import tqdm
# # 2. Prepare data
# dataset_dir = image_path  # subfolders per person
# embeddings, labels = [], []

# for person in tqdm.tqdm(os.listdir(dataset_dir)):
#     person_dir = os.path.join(dataset_dir, person)
#     if not os.path.isdir(person_dir):
#         continue
#     for img_name in os.listdir(person_dir):
#         img_path = os.path.join(person_dir, img_name)
#         bgr = cv2.imread(img_path)
#         if bgr is None:
#             continue
#         rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
#         # Detect faces
#         rects = hog_detector(rgb, 1)  # upsample once
#         if not rects:
#             continue
#         # Assume first detected face
#         aligned = dlib.get_face_chip(rgb, shape_pred(rgb, rects[0]), size=150)
#         # Compute embedding
#         vec = np.array(face_rec.compute_face_descriptor(aligned, num_jitters=1))
#         embeddings.append(vec)
#         labels.append(person)


# **2.  Using ArcFace/Insightface/AntelopeV2**

In [13]:
!wget https://sourceforge.net/projects/insightface.mirror/files/v0.7/antelopev2.zip

--2025-04-28 18:17:30--  https://sourceforge.net/projects/insightface.mirror/files/v0.7/antelopev2.zip
Resolving sourceforge.net (sourceforge.net)... 104.18.12.149, 104.18.13.149, 2606:4700::6812:d95, ...
Connecting to sourceforge.net (sourceforge.net)|104.18.12.149|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sourceforge.net/projects/insightface.mirror/files/v0.7/antelopev2.zip/ [following]
--2025-04-28 18:17:30--  https://sourceforge.net/projects/insightface.mirror/files/v0.7/antelopev2.zip/
Reusing existing connection to sourceforge.net:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sourceforge.net/projects/insightface.mirror/files/v0.7/antelopev2.zip/download [following]
--2025-04-28 18:17:30--  https://sourceforge.net/projects/insightface.mirror/files/v0.7/antelopev2.zip/download
Reusing existing connection to sourceforge.net:443.
HTTP request sent, awaiting response... 302 Found
Location: 

In [15]:
import zipfile

def unzip_file(zip_file_path, extract_to_path):
    """Unzips a zip file to a specified directory.

    Args:
        zip_file_path (str): The path to the zip file.
        extract_to_path (str): The path to the directory where the contents
                                 will be extracted.
    """
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to_path)
        print(f"Successfully extracted '{zip_file_path}' to '{extract_to_path}'")
    except FileNotFoundError:
        print(f"Error: Zip file not found at '{zip_file_path}'")
    except zipfile.BadZipFile:
        print(f"Error: '{zip_file_path}' is not a valid zip file.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
zip_file = '/content/antelopev2.zip'  # Replace with the actual path to your zip file
extract_dir = '/content/drive/MyDrive/FaceRecognitionModels' # Replace with the desired extraction directory

# unzip_file(zip_file, extract_dir)

Successfully extracted '/content/antelopev2.zip' to '/content/drive/MyDrive/FaceRecognitionModels'


In [32]:
!mkdir -p /content/drive/MyDrive/FaceRecognitionModels/antelopev2/detection
!mkdir -p /content/drive/MyDrive/FaceRecognitionModels/antelopev2/recognition

!mv /content/drive/MyDrive/FaceRecognitionModels/antelopev2/scrfd_10g_bnkps.onnx /content/drive/MyDrive/FaceRecognitionModels/antelopev2/detection/
!mv /content/drive/MyDrive/FaceRecognitionModels/antelopev2/glintr100.onnx /content/drive/MyDrive/FaceRecognitionModels/antelopev2/recognition/


In [36]:
from insightface.app import FaceAnalysis
from insightface.model_zoo import model_zoo

# 1. Manually load detection model
det_model = model_zoo.get_model("/content/drive/MyDrive/FaceRecognitionModels/antelopev2/detection/scrfd_10g_bnkps.onnx", task='detection')

# 2. Manually load recognition model
rec_model = model_zoo.get_model("/content/drive/MyDrive/FaceRecognitionModels/antelopev2/recognition/glintr100.onnx", task='recognition')

# 3. Now load FaceAnalysis
app = FaceAnalysis()
app.det_model = det_model
app.rec_model = rec_model
app.models = {'detection': det_model, 'recognition': rec_model}

# 4. Prepare (ctx_id=0 means GPU, ctx_id=-1 means CPU)
app.prepare(ctx_id=0, det_size=(640, 640))


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse

100%|██████████| 281857/281857 [00:05<00:00, 52951.60KB/s]


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

In [37]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# 2. Prepare data with tqdm bars
dataset_dir = image_path    # subfolders per person
embeddings, labels = [], []

for person in tqdm(os.listdir(dataset_dir), desc="Persons", unit="person"):
    person_dir = os.path.join(dataset_dir, person)
    if not os.path.isdir(person_dir):
        continue

    for img_name in tqdm(os.listdir(person_dir),
                         desc=f"Images ({person})", unit="img", leave=False):
        img_path = os.path.join(person_dir, img_name)
        bgr = cv2.imread(img_path)
        if bgr is None:
            continue

        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        # 3. Detect & align & embed in one go
        faces = app.get(rgb)  # returns list of Face objects :contentReference[oaicite:5]{index=5}
        if not faces:
            continue

        # 4. Take the first face
        embedding = faces[0].embedding  # a 512-dim NumPy array :contentReference[oaicite:6]{index=6}
        embeddings.append(embedding)
        labels.append(person)


Persons: 100%|██████████| 9/9 [00:45<00:00,  5.08s/person]


# **Modelling**

In [38]:
embeddings = np.array(embeddings)
labels = np.array(labels)

In [39]:
import pandas as pd
data = pd.DataFrame(embeddings)

In [40]:
combined = pd.concat([data,pd.Series(labels, name='label')],axis=1)

In [41]:
from sklearn.utils import shuffle
combined = shuffle(combined).reset_index(drop=True)

In [42]:
combined.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,label
0,-0.375179,1.729687,1.020849,-0.521455,0.238787,0.529591,0.051781,0.641301,-0.701668,0.951308,...,0.144872,-0.450478,0.292580,0.100530,-0.244754,1.572381,1.281895,-1.894132,0.393078,Gringo
1,-0.931209,-1.557596,-1.295877,-0.854050,-1.310941,-1.506912,0.287071,1.141141,0.710364,1.857338,...,0.534868,0.437685,0.307075,0.511756,1.645488,1.409317,0.600473,-0.794263,-1.330089,Richard
2,0.088669,1.456016,-0.823273,-0.229758,0.532822,-0.100596,1.293430,1.191461,1.056757,-1.089044,...,1.977854,1.727073,0.896384,0.140648,-0.031818,0.748293,0.453764,-0.130092,0.641524,josh
3,0.752287,-0.493572,-1.920135,0.396108,-0.651088,-0.662359,-0.539687,0.103446,0.335007,1.066950,...,-1.608197,-0.524554,-0.275602,-0.217469,0.440505,0.658841,-0.793003,1.286612,4.167459,Elyse
4,-0.567595,-0.064378,-2.188199,1.203913,-0.517041,-0.169671,-0.952663,-0.651906,-1.417024,0.524723,...,-0.380877,-0.346660,0.083812,-0.347702,1.001062,-0.736771,0.575759,1.238185,3.509979,Elyse


In [43]:
from sklearn.preprocessing import LabelEncoder

# Example target
y = combined['label'].tolist()
y

# # Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [44]:
y_encoded

array([2, 7, 8, 1, 1, 6, 5, 7, 1, 8, 4, 0, 2, 7, 3, 7, 3, 8, 1, 8, 4, 8,
       1, 4, 7, 2, 4, 1, 0, 1, 7, 1, 7, 4, 7, 8, 3, 8, 7, 1, 0, 7, 0, 8,
       7, 1, 3, 5, 5, 4, 7, 1, 7, 8, 2, 8, 5, 8, 1, 1, 6, 1])

In [45]:
from sklearn.model_selection import train_test_split as tts

In [46]:
X_train,X_test,y_train,y_test = tts((combined.drop('label', axis = 1)),y_encoded,test_size=0.13,random_state=6)

In [47]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((53, 512), (9, 512), (53,), (9,))

In [48]:
clf = SVC(probability=True)
clf.fit(X_train, y_train)

SVC(probability=True)

In [49]:
predictions = clf.predict(X_test)

In [50]:
predictions

array([7, 7, 2, 6, 7, 4, 1, 8, 1])

In [51]:
y_test

array([7, 7, 2, 6, 7, 4, 1, 8, 1])

In [52]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.8 MB/s eta 0:00:00


In [53]:
from catboost import CatBoostClassifier

In [55]:
model = CatBoostClassifier(
    iterations=100,     # number of trees
    learning_rate=0.1,  # how fast to learn
    depth=6,            # depth of trees
    verbose=0           # set verbose>0 if you want training logs
)

In [56]:
model.fit(X_train,y_train)

In [57]:
model.predict(X_test).flatten().tolist(),model.predict_proba(X_test)

([7, 7, 2, 6, 7, 4, 1, 8, 1],
 array([[0.07931066, 0.12526312, 0.07040454, 0.09613612, 0.0672763 ,
         0.07732052, 0.06515198, 0.33482693, 0.08430984],
        [0.0247456 , 0.02906472, 0.02278437, 0.02237649, 0.02429659,
         0.0273301 , 0.02136271, 0.79396905, 0.03407037],
        [0.09404578, 0.13721932, 0.23362142, 0.09813979, 0.10279342,
         0.09515464, 0.08325608, 0.08788111, 0.06788844],
        [0.08243591, 0.1116429 , 0.09766584, 0.0912031 , 0.0993461 ,
         0.09958203, 0.2116115 , 0.08602306, 0.12048956],
        [0.02127007, 0.02951138, 0.02061026, 0.02193555, 0.02096154,
         0.02334719, 0.01813102, 0.82189394, 0.02233905],
        [0.04360923, 0.05744025, 0.04674359, 0.04232024, 0.61778366,
         0.04899882, 0.04235179, 0.05110941, 0.04964301],
        [0.02696299, 0.76434037, 0.02665193, 0.02760887, 0.02358557,
         0.02749897, 0.02598359, 0.04189588, 0.03547183],
        [0.0248948 , 0.02628364, 0.02188911, 0.02237041, 0.02069215,
         0.0

In [58]:
y_test

array([7, 7, 2, 6, 7, 4, 1, 8, 1])

In [62]:
import pickle
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from insightface.model_zoo import model_zoo
from base64 import b64decode
from google.colab.output import eval_js
from IPython.display import display, Javascript

# ------------- 1) load clf + le --------------
# with open('/content/drive/MyDrive/FaceRecognitionModels/face_rec_model_arcface.pkl','rb') as f:
#     data = pickle.load(f)
# clf = data['clf']
# le  = data['le']

# ------------- 2) setup InsightFace --------------
app = FaceAnalysis()
# (load your ONNX models exactly as you did before)
det = model_zoo.get_model("/content/drive/MyDrive/FaceRecognitionModels/antelopev2/detection/scrfd_10g_bnkps.onnx",
                         task='detection')
rec = model_zoo.get_model("/content/drive/MyDrive/FaceRecognitionModels/antelopev2/recognition/glintr100.onnx",
                         task='recognition')
app.det_model = det
app.rec_model = rec
app.models = {'detection':det, 'recognition':rec}
app.prepare(ctx_id=0, det_size=(640,640))

# ----------- 3) capture photo util -------------
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
            async function takePhoto(quality) {
              const div = document.createElement('div');
              const capture = document.createElement('button');
              capture.textContent = 'Capture';
              div.appendChild(capture);

              const video = document.createElement('video');
              video.style.display = 'block';
              const stream = await navigator.mediaDevices.getUserMedia({video: true});

              document.body.appendChild(div);
              div.appendChild(video);
              video.srcObject = stream;
              await video.play();

              // Resize the output to fit the video element.
              google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

              // Wait for Capture to be clicked.
              await new Promise((resolve) => capture.onclick = resolve);

              const canvas = document.createElement('canvas');
              canvas.width = video.videoWidth;
              canvas.height = video.videoHeight;
              canvas.getContext('2d').drawImage(video, 0, 0);
              stream.getVideoTracks()[0].stop();
              div.remove();
              return canvas.toDataURL('image/jpeg', quality);
            }

     ''')
    display(js)
    data = eval_js(f'takePhoto({quality})')
    img_data = b64decode(data.split(',')[1])
    with open(filename,'wb') as f: f.write(img_data)
    return filename

# ----------- 4) capture & read image -----------
fname = take_photo()
bgr = cv2.imread(fname)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

# ----------- 5) detect & embed face -----------
faces = app.get(rgb)
if not faces:
    print("No face detected")
else:
    emb = faces[0].embedding          # 512-D vector
    # ----------- 6) predict & decode -----------
    y_pred = clf.predict([emb])[0]    # integer, e.g. 2
    name   = le.inverse_transform([y_pred])[0]
    print("I see:", name)


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

<IPython.core.display.Javascript object>

I see: Richard
